In [4]:
# Imports and Utility functions
from scripts.utils import clear_keri, exec
from scripts.saidify import process_schema_file, get_schema_said


# VC IPEX

## Setup holder and verifier

### Holder setup (boilerplate)

This should be familiar by now; we initialize a keystore for the holder of the credential

In [ ]:
holder_keystore_name = "holder_ks"

holder_keystore_passcode = exec("kli passcode generate")

holder_keystore_salt = exec("kli salt")
# Alias for our non-transferable AID
holder_aid = "holder_aid"

# Initialize the keystore
!kli init --name {holder_keystore_name} --passcode {holder_keystore_passcode} --salt {holder_keystore_salt}\
    --config-dir ./config \
    --config-file keystore_init_config.json

!kli incept --name {holder_keystore_name} --alias {holder_aid} --passcode {holder_keystore_passcode} \
    --file ./config/aid_inception_config.json

ERR: Last seed missing or provided last seed not associated with last aeid=BOmCm9hIUqgObUJVDLX0N8wi7DgWlENG7vumiHrcJwnV.
Last seed missing or provided last seed not associated with last aeid=BOmCm9hIUqgObUJVDLX0N8wi7DgWlENG7vumiHrcJwnV.
Valid passcode required, try again...
Passcode: 

### Issuer setup (boilerplate)

We do the same for the issuer

In [ ]:
# Issuer setup
issuer_keystore_name = "issuer_ks"

issuer_keystore_passcode = exec("kli passcode generate")

issuer_keystore_salt = exec("kli salt")
# Alias for our non-transferable AID
issuer_aid = "issuer_aid"

# Initialize the keystore
!kli init --name {issuer_keystore_name} --passcode {issuer_keystore_passcode} --salt {issuer_keystore_salt}\
    --config-dir ./config \
    --config-file keystore_init_config.json

!kli incept --name {issuer_keystore_name} --passcode {issuer_keystore_passcode} --alias {issuer_aid}  \
    --file ./config/aid_inception_config.json

### Issuer/Holder OOBI 

Now, the issuer and holder need to establish a connection by resolving OOBIS. (This time we skip the challenge/reponse for convenience)

In [ ]:
holder_oobi_gen = f"kli oobi generate --name {holder_keystore_name} --alias {holder_aid} --passcode {holder_keystore_passcode} --role witness"
holder_oobi = exec(holder_oobi_gen)

issuer_oobi_gen = f"kli oobi generate --name {issuer_keystore_name} --alias {issuer_aid} --passcode {issuer_keystore_passcode} --role witness"
issuer_oobi = exec(issuer_oobi_gen)

!kli oobi resolve --name {holder_keystore_name} --passcode {holder_keystore_passcode} --oobi-alias {issuer_aid} \
    --oobi {issuer_oobi}

!kli oobi resolve --name {issuer_keystore_name} --passcode {issuer_keystore_passcode} --oobi-alias {holder_aid}\
    --oobi {holder_oobi}

### Issuer Registry

We need to setup a credential registry for the issuer.

In [ ]:
issuer_registry_name="issuer_registry"

!kli vc registry incept --name {issuer_keystore_name} --passcode {issuer_keystore_passcode} \
    --registry-name {issuer_registry_name} \
    --alias {issuer_aid}

To query the status of a registry use the command `kli vc registry status`

In [ ]:
!kli vc registry status --name {issuer_keystore_name} --passcode {issuer_keystore_passcode} --registry-name {issuer_registry_name} 

## Schema preparation

As seen before we need a schema to issue credentials. 

For this example we have prepared an schema to simulate an access pass for an event. It has the following attributes:
```json
"eventName": {
  "description": "The event name",
  "type": "string"
},
"accessLevel": {
  "description": "staff/speaker/attendee",
  "type": "string",
  "enum": [
    "staff",
    "speaker",
    "attendee"
  ]
},
"validDate": {
  "description": "Valid date yyyy-mm-dd",
  "type": "string"
}
```

To see the full schema click **[here](config/schemas/event_pass_schema.bak.json)**.  

You may have noticed the schema is not saidified, so let's do that:

In [ ]:
process_schema_file("config/schemas/event_pass_schema.bak.json", "config/schemas/event_pass_schema.json") 

schema_said = get_schema_said("config/schemas/event_pass_schema.json")

print(schema_said)


### vLEI-server
After the schema has been "saidified," (see the "saidified" schema **[here](config/schemas/event_pass_schema.json)**) it needs to be made available for any party needing it. In the case of the issuer and the holder. 
For this purpose, we have embedded a pre-configured service that already has the saidified schema pre-loaded.

you can query the schema by using the pre-defined schema oobi.

🚧 explain a little bit about vLEI-Server
note: we will expand on the vLEI-server later on



In [ ]:
!curl -s http://vlei-server:7723/oobi/{schema_said}

### Resolve Schema oobi

The issuer and holder need to resolve the schema by using it's oobi

In [ ]:
schema_oobi_alias = "schema_oobi"
schema_oobi = f"http://vlei-server:7723/oobi/{schema_said}"

!kli oobi resolve --name {holder_keystore_name} --passcode {holder_keystore_passcode} --oobi-alias {schema_oobi_alias} \
    --oobi {schema_oobi}

!kli oobi resolve --name {issuer_keystore_name} --passcode {issuer_keystore_passcode} --oobi-alias {schema_oobi_alias}\
    --oobi {schema_oobi}

## Prepare and Create Credential

We have our issuer and verifier and schema ready. now we need to create the credential 

### Prepare credential data

To create the credential we need to provide the values corresponding to the attributes. This values are fed into the `kli command` using a file
pre-define, see file

In [ ]:
!cat config/credential_data/event_pass_cred_data.json

### Create credential

To create the credential, use the command `kli vc create`. You provide the issuer keystore parameters, the alias of the AID that will act as the credential issuer, and the `--registry` to manage the credential. the `--schema` said (remember the schema was loaded from the vLEI-server), the `--recipient` is the credential subject (also called the issue), `--data` are the attributes for this particular credential, and `--time` is the timestamp for the credential creation 

In [ ]:
time = exec("kli time")

!kli vc create --name {issuer_keystore_name} --passcode {issuer_keystore_passcode} \
    --alias {issuer_aid} \
    --registry-name {issuer_registry_name} \
    --schema {schema_said} \
    --recipient {holder_aid} \
    --data "@./config/credential_data/event_pass_cred_data.json" \
    --time {time}

### Display issued credential

The command `kli vc list` allows you to see the issued credential

In [ ]:
!kli vc list  --name {issuer_keystore_name} --passcode {issuer_keystore_passcode} --issued --verbose

### Get credential SAID

We need to retrieve the credential said, to use it as credential identifier in the next steps.
to achieve that we use the `kli vc list` again put addinf additional parameters `--said` to return only the SAID of found credentials

In [ ]:
get_credential_said = f"kli vc list --name {issuer_keystore_name} --passcode {issuer_keystore_passcode} --alias {issuer_aid} --issued --said --schema {schema_said}"
credential_said=exec(get_credential_said)

print(credential_said)

## Ipex Credetial issuance

At this point, the credential has been created. but it remains with the issuer. To send the credential, we need to use the IPEX protocol. 

### Issuer initiate Ipex exchange (credential offer)

The `kli ipex grant` command is used to deliver the credential. Here, the language about the management of the credential is sometimes referred to as the discloser and the disclosee. The issuer (discloser) is disclosing the credential to the holder (disclose)
Using `kli ipex grant`, the discloser sends a message with the credential.

The parameters:
- `--said` is the credential identifier value we just retrieved
- `--recipient` in this case is the holder
- `--time` is a timestamp

In [ ]:
time = exec("kli time")

!kli ipex grant \
    --name {issuer_keystore_name} \
    --passcode {issuer_keystore_passcode} \
    --alias {issuer_aid} \
    --said {credential_said} \
    --recipient {holder_aid} \
    --time {time}

### Issuer check sent message

This step is not required for the workflow, but it allows you see a little bit of the inner workings

In [ ]:
!kli ipex list --name {issuer_keystore_name} \
    --passcode {issuer_keystore_passcode} \
    --alias {issuer_aid} \
    --sent

### Holder poll mailbox and get message said (IPEX Message)

The holder needs to check the mailbox for the ipex message containing a credential 🚩

In [ ]:
get_ipex_said=f"kli ipex list --name {holder_keystore_name} --passcode {holder_keystore_passcode} --alias {holder_aid} --poll --said"
ipex_said=exec(get_ipex_said)

print(ipex_said)


### Holder check received message

This step is not required for the workflow, but it allows you see a little bit of the inner workings

In [ ]:
!kli ipex list --name {holder_keystore_name} \
    --passcode {holder_keystore_passcode} \
    --alias {holder_aid} 

### Holder accept credential 

Now the we have the ipex message said, the holder can admit the message

In [ ]:
time = exec("kli time")

!kli ipex admit \
    --name {holder_keystore_name} \
    --passcode {holder_keystore_passcode} \
    --alias {holder_aid} \
    --said {ipex_said} \
    --time {time}



### Holder check sent message

not required but we can see the ADMIT message sent

In [ ]:
!kli ipex list --name {holder_keystore_name} \
    --passcode {holder_keystore_passcode} \
    --alias {holder_aid} \
    --sent

### Holder displays credential

The holder now has the credential.

In [ ]:
!kli vc list  --name {holder_keystore_name} --passcode {holder_keystore_passcode} --verbose

# Issuing an ACDC

**Objective:** Learn how to create a Credential Registry and issue a basic Authentic Chained Data Container (ACDC) using the KERI command-line interface (`kli`).

*(Assumed Knowledge: You should understand ACDCs and Schemas and have a SAIDified schema file ready, e.g., `sample_credential_schema.json` with its calculated SAID.)*

## Credential Registries (TELs)

Before an issuer can start creating ACDCs, it needs a way to manage its issuance and revocation status. In KERI, this is handled by a **Credential Registry**.

Think of a registry as a dedicated ledger within the issuer's environment, specifically for tracking the credentials they issue. Under the hood, this registry uses one or more **Transaction Event Logs (TELs)**.

* **What is a TEL?** A TEL is another secure, append-only log, similar to a KEL (Key Event Log). However, instead of tracking key events for an identifier (AID), a TEL tracks events related to credentials issued under that AID, such as issuance (`iss`) and revocation (`rev`).
* **Anchoring:** Events in a TEL are cryptographically anchored back to the issuer's main KEL. This ensures that the credential's status is tied to the issuer's control authority.
* **Management TEL:** When you create a registry, a special *management* TEL is created first. This management TEL tracks the overall registry configuration. Later, when individual credentials are issued, they get their *own* TELs to track their specific status, linked back to the management TEL.



Create 2 keystores
Create a registry on each keystore

concepts:  
- credential registry
- TEL A transaction event log is a log of all events occurring for a verifiable credential or a credential registry
    - Management TEL:
        - registrars (backers) 
    - VC TEL: 
- event icp
- event ixn (registry creation)
- Backers
- Public Transaction Event Logs (PTEL):
    - use TEL to connect to connect ACDC with KEL
    - Credential lifecycle (issuance and revocation)  
- and the Issuance and Presentation Exchange (IPEX) protocols